In [1]:
import pandas as pd
import numpy as np
from datetime import date

from ctgan import CTGANSynthesizer
from table_evaluator import load_data, TableEvaluator

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
features_data = pd.read_csv("features_data.csv")
equity_value_data = pd.read_csv("equity_value_data.csv")

In [3]:
'''Function to extract date in %%y-%%m-%%d'''
def todate(timestamp):
    
    s = timestamp.split("-")
    year = s[0]
    month = s[1]
    date = s[2] 
    date = year+'-'+month+'-'+date[0]+date[1]
    return date


'''Function to get the difference between the dates'''
def date_diff(date_,date1_):
    
    d = date_.split("-")
    d_ = date1_.split("-")
    
    d0 = date(int(d[0]), int(d[1]), int(d[2]))
    d1 = date(int(d_[0]), int(d_[1]), int(d_[2]))
    delta = d1 - d0
    return delta.days


In [4]:
'''To create date column in equity_value_data'''
equity_value_data['date'] = equity_value_data['timestamp'].apply(todate)
equity_value_data.head()

,timestamp,close_equity,user_id,date
0,2016-11-16T00:00:00Z,48.16,bcef4fa9b0bdf22bcf7deae708decf03,2016-11-16
1,2016-11-17T00:00:00Z,48.16,bcef4fa9b0bdf22bcf7deae708decf03,2016-11-17
2,2016-11-18T00:00:00Z,48.16,bcef4fa9b0bdf22bcf7deae708decf03,2016-11-18
3,2016-11-21T00:00:00Z,48.16,bcef4fa9b0bdf22bcf7deae708decf03,2016-11-21
4,2016-11-22T00:00:00Z,48.16,bcef4fa9b0bdf22bcf7deae708decf03,2016-11-22


In [5]:
%%time

##############################################################################################################
#####################  Calculating Churn user by applying 28 consecutive days condition  #####################
##############################################################################################################

cnt = 0

#To save Churn users
churn_user = []

for i in range(len(equity_value_data['user_id'])-1):
    
    if equity_value_data['user_id'][i] == equity_value_data['user_id'][i+1]:
        
        days = date_diff(equity_value_data['date'][i],equity_value_data['date'][i+1])
        
        if days >=28 and (equity_value_data['user_id'][i] not in churn_user):
            
            churn_user.append(equity_value_data['user_id'][i])
            cnt += 1

            
Total_User = len(features_data['user_id'])

print("Total user: ", Total_User)
print("Total churn user: ", cnt)
print("Percentage: ",round(cnt / Total_User * 100, 2),"%")

Total user:  5584
Total churn user:  279
Percentage:  5.0 %
Wall time: 23.6 s


In [7]:
'''Adding Churn feature in the features_data'''

features_data['Churn'] = 0
for i in range(len(features_data['user_id'])):
    if features_data['user_id'][i] not in churn_user:
        features_data['Churn'][i] = 'No'
    else:
        features_data['Churn'][i] = 'Yes'     

In [8]:
data = features_data.copy()
data.head()

,risk_tolerance,investment_experience,liquidity_needs,platform,time_spent,instrument_type_first_traded,first_deposit_amount,time_horizon,user_id,Churn
0,high_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,33.129417,stock,40.0,med_time_horizon,895044c23edc821881e87da749c01034,No
1,med_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,16.573517,stock,200.0,short_time_horizon,458b1d95441ced242949deefe8e4b638,No
2,med_risk_tolerance,limited_investment_exp,very_important_liq_need,iOS,10.008367,stock,25.0,long_time_horizon,c7936f653d293479e034865db9bb932f,No
3,med_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,1.031633,stock,100.0,short_time_horizon,b255d4bd6c9ba194d3a350b3e76c6393,No
4,high_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,8.187250,stock,20.0,long_time_horizon,4a168225e89375b8de605cbc0977ae91,No


In [9]:
data.drop('user_id', axis = 1, inplace= True)

In [10]:
categorical_features = ['risk_tolerance', 'investment_experience', 'liquidity_needs',
       'platform', 'instrument_type_first_traded', 'time_horizon', 'Churn']

In [11]:
ctgan = CTGANSynthesizer(verbose=True)
ctgan.fit(data, categorical_features, epochs = 100)

Epoch 1, Loss G:  1.3516, Loss D: -0.1437
Epoch 2, Loss G:  1.1147, Loss D: -0.5408
Epoch 3, Loss G:  0.5772, Loss D: -0.3705
Epoch 4, Loss G:  0.5602, Loss D: -0.2406
Epoch 5, Loss G:  0.7514, Loss D: -0.0292
Epoch 6, Loss G:  0.9225, Loss D: -0.1064
Epoch 7, Loss G:  0.6510, Loss D: -0.0281
Epoch 8, Loss G:  0.6055, Loss D:  0.0672
Epoch 9, Loss G:  0.5385, Loss D:  0.1703
Epoch 10, Loss G:  0.5306, Loss D: -0.0354
Epoch 11, Loss G:  0.5784, Loss D: -0.1107
Epoch 12, Loss G:  0.2695, Loss D: -0.0701
Epoch 13, Loss G:  0.2130, Loss D:  0.1138
Epoch 14, Loss G:  0.2568, Loss D: -0.0003
Epoch 15, Loss G:  0.4089, Loss D:  0.0390
Epoch 16, Loss G:  0.4213, Loss D: -0.0607
Epoch 17, Loss G:  0.5184, Loss D:  0.0177
Epoch 18, Loss G:  0.3438, Loss D:  0.1486
Epoch 19, Loss G:  0.1660, Loss D: -0.0632
Epoch 20, Loss G: -0.0663, Loss D:  0.0634
Epoch 21, Loss G: -0.2298, Loss D:  0.1459
Epoch 22, Loss G: -0.0698, Loss D:  0.0601
Epoch 23, Loss G:  0.0209, Loss D: -0.0523
Epoch 24, Loss G: -0

In [12]:
conditions = {"Churn": 1}

In [13]:
samples = ctgan.sample(5584,condition_column=conditions)

In [14]:
print(samples["Churn"].value_counts())

No     5156
Yes     428
Name: Churn, dtype: int64


In [15]:
print(data["Churn"].value_counts())

No     5305
Yes     279
Name: Churn, dtype: int64


In [16]:
from table_evaluator import load_data, TableEvaluator

In [17]:
table_evaluator =  TableEvaluator(data, samples, cat_cols= categorical_features)

In [18]:
table_evaluator.evaluate(target_col = 'Churn')


Classifier F1-scores and their Jaccard similarities::
                             f1_real  f1_fake  jaccard_similarity
index                                                            
DecisionTreeClassifier_fake   0.8299   0.8496              0.6835
DecisionTreeClassifier_real   0.8800   0.8747              0.7688
LogisticRegression_fake       0.9266   0.9266              1.0000
LogisticRegression_real       0.9382   0.9373              0.9982
MLPClassifier_fake            0.9266   0.9266              1.0000
MLPClassifier_real            0.9382   0.9382              1.0000
RandomForestClassifier_fake   0.9114   0.9176              0.9528
RandomForestClassifier_real   0.9311   0.9293              0.9683

Privacy results:
                                           result
Duplicate rows between sets (real/fake)  (634, 0)
nearest neighbor mean                      0.6271
nearest neighbor std                       0.8045

Miscellaneous results:
                                  Result
Co

In [22]:
samples.to_csv('synthetic.csv', sep='\t')